***Recap**:

Score Function: $s=\sum(x,W)=Wx$

Loss on a single example:  $L_i = \sum_{j\neq y_i} \max(0, s_j - s_{y_i} + \Delta)$

Loss on the whole training data: $L = \frac{1}{N} \sum_i \sum_{j\neq y_i} \left[ \max(0, f(x_i; W)_j - f(x_i; W)_{y_i} + \Delta) \right] + \lambda \sum_k\sum_l W_{k,l}^2$

**Goal**: want  $\nabla _wf (L)$

## Backpropagation
is a way of computing gradients of expressions through recursive application of **chain rule**.

#### Problem Statement
We are given some function f(x)
 where x
 is a vector of inputs and we are interested in computing the gradient of f
 at x
 (i.e. ∇f(x)
 ).

 #### Motivation
 Our target function f will eventually correspond to the loss function L.We condider input examples as constant and the Weights as variable. Our main goal in machine learning is to suit the parameter W as perfectly as possible so that our model performs well. We find this optimized weight through upadates. These updates are done by calculating gradient using  **Backpropagation** algorithm.

 **NB**: Although we don't calculate gradients of input datas. But it can be done and sometimes useful to see what NN might be doing.

 ### Simple expressions and interpretation of the gradient
 **Multiplication**: $f(x,y) = x y \hspace{0.5in} \rightarrow \hspace{0.5in} \frac{\partial f}{\partial x} = y \hspace{0.5in} \frac{\partial f}{\partial y} = x$

 **intuition**:the derivative on each variable tells you the sensitivity of the whole expression on its value. For example, $if x=4,y=−3$
 then $f(x,y)=−12$
 and the derivative on $ x\frac{\partial f}{\partial x} = -3$
. This tells us that if we were to increase the value of this variable by a tiny amount, the effect on the whole expression would be to decrease it (due to the negative sign), and by three times that amount.

**Gradient**:As mentioned, the gradient ∇f
 is the vector of partial derivatives, so we have that $ \nabla f = [\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}] = [y, x]$

 **Addition** $f(x,y) = x + y \hspace{0.5in} \rightarrow \hspace{0.5in} \frac{\partial f}{\partial x} = 1 \hspace{0.5in} \frac{\partial f}{\partial y} = 1$

 **Max gate(gradient router**:) $f(x,y) = \max(x, y) \hspace{0.5in} \rightarrow \hspace{0.5in} \frac{\partial f}{\partial x} = \mathbb{1}(x >= y) \hspace{0.5in} \frac{\partial f}{\partial y} = \mathbb{1}(y >= x)$

### Compound expressions with chain rule (Backpropagation Intuition)
#### Computational Graphs
A computational graph is a way to represent a math function in the language of graph theory. Recall the premise of graph theory: nodes are connected by edges, and everything in the graph is either a node or an edge.

** Graph Node**: In simpler terms , every nodes on the computational graph performs math operations on its input(s) and results an output. We can calculate the **local gradient** of each node (change of the output of a node w.r.t. its input(s)). This local gradients will be useful in backporpagation which we are going to see in later section.

**Why we use computational graph**?: In neural network eventually we oupts predictions based on many complex function operations. It is really tough to formulate them. On the other hand its easy to represent them in graphs. Additionally, using this compoutational graph it's easy to calculate **local gradients**.

For example, consider the relatively simple expression: $f(x, y, z) = (x + y) * z$.This expression can be broken down into two expressions: $q=x+y$
 and $f=qz$. According to demonstration given above there are two operational node. We can find derivatives(local gradients) of these two node w.r.t their input(s) on output separtely.

 f is just multiplication of q and z, so $\frac{\partial f}{\partial q} = z, \frac{\partial f}{\partial z} = q$, and q
 is addition of x and y so $\frac{\partial q}{\partial x} = 1$, $\frac{\partial q}{\partial y} = 1$


![alt text](Images/cg.png "computational graph")


***ULTIMATE GUIDE***: Every node computes two things--
          - ouput
          - loacal gradient(the change of its output w.r.t. its input(s))
    However, once the forward pass is over, during backpropagation the gate will eventually learn about the gradient of its output value on the final output of the entire circuit. Chain rule says that the gate should take that gradient and multiply it into every gradient it normally computes for all of its inputs (**local gradient** X **upstream gradietnt**).




Lets see with an example

In [1]:
# set some inputs
x = -2; y = 5; z = -4

# perform the forward pass
q = x + y # q becomes 3
f = q * z # f becomes -12

# perform the backward pass (backpropagation) in reverse order:
# first backprop through f = q * z
dfdz = q # df/dz = q, so gradient on z becomes 3
dfdq = z # df/dq = z, so gradient on q becomes -4
dqdx = 1.0
dqdy = 1.0
# now backprop through q = x + y
dfdx = dfdq * dqdx  # The multiplication here is the chain rule!
dfdy = dfdq * dqdy

We are left with the gradient in the variables [dfdx,dfdy,dfdz], which tell us the sensitivity of the variables x,y,z on f!. This is the simplest example of backpropagation

### Modularity: Sigmoid example:
Any kind of differentiable function can act as a gate, and we can group multiple gates into a single gate, or decompose a function into multiple gates whenever it is convenient. Lets look at another expression that illustrates this point: $f(w,x) = \frac{1}{1+e^{-(w_0x_0 + w_1x_1 + w_2)}}$ . say , $f(w,x) = \frac{1}{1+e^{-(w_0x_0 + w_1x_1 + w_2)}}$ this part is computed in one node. We can then consider another node(sigmoid node) that will take this as inut and perfomrs sigmoid opearation. or we can decompose the function of the sigmoid node inot multiple nodes.

![alt text](Images/scg.PNG "sigmoid functions")

#### derivative of sigmoid function($\sigma(x)(1-\sigma(x))$):
$\begin{align}
\frac{d}{dx}\sigma(x) & = \frac{d}{dx} \frac{1}{1+e^{-x}}\\
& = \frac{d}{dx}\big( 1+ e^{-x} \big) ^{-1} \quad[\text{apply chain rule}]\\
& = -(1 + e^{-x})^{-2} \cdot \frac{d}{dx}(1+e^{-x}) \quad[\text{apply sum rule}] \\
& = -(1 + e^{-x})^{-2} \cdot \bigg(\frac{d}{dx}1 + \frac{d}{dx}e^{-x}\bigg) \\
& = -(1 + e^{-x})^{-2} \cdot \frac{d}{dx}e^{-x} \quad[\text{apply chain rule}]\\
& = -(1 + e^{-x})^{-2} \cdot e^{-x}\frac{d}{dx} (-x)\\
& = -(1 + e^{-x})^{-2} \cdot \big(- e^{-x} \big)\\
& = \frac{1}{(1 + e^{-x})^{2}} \cdot e^{-x} \\
& = \frac{e^{-x}}{(1 + e^{-x})^{2}}
\end{align}$

We can further simplify the derivative to the expression $\sigma(x)(1-\sigma(x))$
$\begin{align}
 \frac{e^{-x}}{(1 + e^{-x})^{2}} &= \frac{e^{-x}}{1 + e^{-x}}\cdot\frac{1}{1 + e^{-x}}  \\
& = \frac{-1 + 1 + e^{-x}}{1 + e^{-x}}\cdot\frac{1}{1 + e^{-x}}\\
& = \bigg(\frac{-1 }{1 + e^{-x}} + \frac{1 + e^{-x} }{1 + e^{-x}} \bigg)\cdot\frac{1}{1 + e^{-x}}\\
& = \bigg(\frac{-1 }{1 + e^{-x}} + 1 \bigg)\cdot\frac{1}{1 + e^{-x}}\\
& = \bigg(1 - \frac{1 }{1 + e^{-x}}  \bigg)\cdot\frac{1}{1 + e^{-x}}\\
& = \big(1-\sigma(x)\big) \cdot \sigma(x)
\end{align}$


### Backpropagation of sigmoid neuron

In [4]:
import math
w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function

# backward pass through the neuron (backpropagation)
ddot = (1 - f) * f # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot] # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot] # backprop into w   1.0*ddot- for bias term
# we're done! we have the gradients on the inputs to the circuit

### Backprop in practice: Staged computation
$f(x,y) = \frac{x + \sigma(y)}{\sigma(x) + (x+y)^2}$  computing its gradient is really a daunting task. But using staged computation and backpropagation its not that hard.

In [6]:
##Forward pass

x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!                                 #(8)

 by the end of the expression we have computed the forward pass. Notice that we have structured the code in such way that it contains multiple intermediate variables, each of which are only simple expressions for which we already know the local gradients.

In [8]:
# backprop f = num * invden
dnum = invden # gradient on numerator                             #(8)
dinvden = num                                                     #(8)
# backprop invden = 1.0 / den 
dden = (-1.0 / (den**2)) * dinvden                                #(7)
# backprop den = sigx + xpysqr
dsigx = (1) * dden                                                #(6)
dxpysqr = (1) * dden                                              #(6)
# backprop xpysqr = xpy**2
dxpy = (2 * xpy) * dxpysqr                                        #(5)
# backprop xpy = x + y
dx = (1) * dxpy                                                   #(4)
dy = (1) * dxpy                                                   #(4)
# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx # Notice += !! See notes below  #(3)
# backprop num = x + sigy
dx += (1) * dnum                                                  #(2)
dsigy = (1) * dnum                                                #(2)
# backprop sigy = 1.0 / (1 + math.exp(-y))
dy += ((1 - sigy) * sigy) * dsigy                                 #(1)
# done! phew

#### Cache forward pass variables:
To compute the backward pass it is very helpful to have some of the variables that were used in the forward pass. In practice you want to structure your code so that you cache these variables, and so that they are available during backpropagation. If this is too difficult, it is possible (but wasteful) to recompute them.

#### Gradients add up at forks:
The forward expression involves the variables x,y multiple times, so when we perform backpropagation we must be careful to use += instead of = to accumulate the gradient on these variables (otherwise we would overwrite it). This follows the multivariable chain rule in Calculus, which states that if a variable branches out to different parts of the circuit, then the gradients that flow back to it will add.

### Gradients for vectorized operations
Matrix-Matrix multiply gradient. Possibly the most tricky operation is the matrix-matrix multiplication (which generalizes all matrix-vector and vector-vector) multiply operations:


In [10]:
# forward pass
import numpy as np
W = np.random.randn(5, 10)
X = np.random.randn(10, 3)
D = W.dot(X)

# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) # same shape as D
dW = dD.dot(X.T) #.T gives the transpose of the matrix
dX = W.T.dot(dD)